In [90]:
import tensorflow as tf
import pickle
from colors import ColorsCorpusReader
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL
from pprint import pprint
from tqdm import tqdm
from itertools import product
from math import cos,sin,pi
from sklearn.model_selection import train_test_split
import numpy as np
import re
from tensorflow.keras.layers import GRU,Embedding,Dense,Input
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras import Model
utils.fix_random_seeds()
from itertools import permutations 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
tf.__version__

'2.1.1-dlenv_tfe'

In [92]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

def randvec(w, n=300, lower=-1.0, upper=1.0):
    """Returns a random vector of length `n`. `w` is ignored."""
    return utils.randvec(n=n, lower=lower, upper=upper)

def load_glove300():
    glove_src = os.path.join(GLOVE_HOME, 'glove.6B.300d.txt')
    # Creates a dict mapping strings (words) to GloVe vectors:
    GLOVE = utils.glove2dict(glove_src)
    return GLOVE

GLOVE = load_glove300()

def glove_vec(w):
    """Return `w`'s GloVe representation if available, else return
    a random vector."""
    return GLOVE.get(w, randvec(w, n=300))

In [93]:
MAX_LEN=15
HIDDEN_SIZE=200
EMBEDDING_SIZE=300
COLOR_REP_SIZE=54
BATCH_SIZE=512
MAX_STEPS=10

In [94]:
def formatData(examples):
    colors=[]
    desc=[]

    for ex in examples:
        colors.append(ex.colors)
        txt=ex.parse_turns()
        desc.append(" , ".join(txt))
    return colors,desc

In [95]:
x=[0,1,2]
permutations=list(product(x,x,x))

def represent_color_context(colors):
    return [represent_color(color) for color in colors]

def represent_color(color):
    H=color[0]
    L=color[1]
    S=color[2]
    H_new=H
    V=L+(S*min(L,1-L))
    if V==0:
        S_new=0
    else:
        S_new= 2* (1-L/V)
        
    color=[round(H_new,2),round(S_new,2),round(V,2)]
    real_features=[]
    img_features=[]
    
    for i in permutations:
        real=cos(2*pi*(color[0]*i[0]+color[1]*i[1]+color[2]*i[2]))
        imaginary=-sin(2*pi*(color[0]*i[0]+color[1]*i[1]+color[2]*i[2]))
        real_features.append(real)
        img_features.append(imaginary)
    return real_features+img_features

In [96]:
def applyFourierTransfrom(colors):
    fourier_colors=[]

    for ex in colors:
        rep=represent_color_context(ex)
        fourier_colors.append(rep)
    
    return np.array(fourier_colors)

In [97]:
def getVocab(desc):
    counter=[]
    for txt in desc:
        txt=txt.strip()
        txt=re.sub("\s+"," ",txt)
        txt=txt.lower()
        toks=txt.split(" ")
        counter.extend(toks)
    df=pd.DataFrame(pd.value_counts(pd.Series(counter))).reset_index()
    df.columns=['token','rep']
    
    return df.token[df.rep>1].tolist()

In [98]:
def getInputAndTargetSeqs(seqs):
    inputs=[]
    outputs=[]
    for seq in seqs:
        inputs.append(seq[:-1])
        outputs.append(seq[1:])
    inputs=tf.keras.preprocessing.sequence.pad_sequences(inputs,maxlen=MAX_LEN,padding="post")
    outputs=tf.keras.preprocessing.sequence.pad_sequences(outputs,maxlen=MAX_LEN,padding="post")
    return inputs,outputs


In [99]:
def tokenizeSeqs(seqs):
    output=[]
    for i in seqs:
        i=i.strip()
        i=re.sub("\s+"," ",i)
        i=i.lower()
        toks=i.split(" ")
        ids=[tok2id[START_SYMBOL]]+[tok2id[x] if x in tok2id.keys() else tok2id[UNK_SYMBOL] for x in toks  ]+[tok2id[END_SYMBOL]]
        output.append(ids)
    return output

In [100]:
def getEmbeddingMatrix(id2tok,EMBEDDING_SIZE):
    embedding_matrix = np.zeros((len(id2tok), EMBEDDING_SIZE))
    for i in id2tok.keys():
        embedding_matrix[i]=glove_vec(id2tok[i])
    return embedding_matrix

In [101]:
# def createModel(HIDDEN_SIZE,EMBEDDING_SIZE,VOCAB_SIZE):
#     color_input=Input(shape=(None,54))
#     text_input = Input(shape=(None,))
#     color_encoder=GRU(HIDDEN_SIZE, return_state=True)
#     text_decoder = GRU(HIDDEN_SIZE, return_sequences=True)
#     embedding_layer=Embedding(VOCAB_SIZE,EMBEDDING_SIZE,mask_zero=True)
#     output_layer=Dense(VOCAB_SIZE)
    
#     encoder_outputs, state_h = color_encoder(color_input)
#     embeddings=embedding_layer(text_input)
#     decoder_output = text_decoder(embeddings, initial_state=state_h)
#     final_outputs = output_layer(decoder_output)
#     model = Model([color_input,text_input],final_outputs)
#     return model

def createModel(HIDDEN_SIZE,EMBEDDING_SIZE,VOCAB_SIZE,COLOR_REP_SIZE,embedding_matrix):
    color_input=Input(shape=(None,COLOR_REP_SIZE,))
    text_input = Input(shape=(None,))
    
    color_encoder=GRU(HIDDEN_SIZE, return_state=True)
    text_decoder = GRU(HIDDEN_SIZE, return_sequences=True)
    embedding_layer=Embedding(VOCAB_SIZE,EMBEDDING_SIZE,mask_zero=True,weights=[embedding_matrix])
    output_layer=Dense(VOCAB_SIZE)

    encoder_outputs, state_h = color_encoder(color_input)
    embeddings=embedding_layer(text_input)
    
    last_color_representation=tf.reshape(color_input,(-1,COLOR_REP_SIZE*3))[:,COLOR_REP_SIZE*2:]
    last_color_representation=tf.expand_dims(last_color_representation,axis=1)
    last_color_representation=tf.repeat(last_color_representation,repeats=MAX_LEN,axis=1)

    
    new_embeddings=tf.concat([embeddings,last_color_representation],axis=2)
    decoder_output = text_decoder(new_embeddings, initial_state=state_h)
    final_outputs = output_layer(decoder_output)
    model = Model([color_input,text_input],final_outputs)
    return model
# x=createModel(HIDDEN_SIZE,EMBEDDING_SIZE,VOCAB_SIZE,COLOR_REP_SIZE)

In [102]:
COLORS_SRC_FILENAME = os.path.join("data", "colors", "filteredCorpus.csv")

corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME,
    word_count=None,
    normalize_colors=True)
examples=list(corpus.read())

In [103]:
train_examples,test_examples = train_test_split(examples,test_size=0.1,random_state=42,shuffle=True)
print("Train Counts : ",(pd.value_counts([x.condition for x in train_examples])))
print("\n")
print("Test Counts : ",pd.value_counts([x.condition for x in test_examples]))

Train Counts :  split    14155
far      14149
close    13990
dtype: int64


Test Counts :  far      1633
split    1538
close    1529
dtype: int64


In [104]:
#get colors and seqs in format
train_colors,train_descs=formatData(train_examples)
train_fourier_colors=applyFourierTransfrom(train_colors)

test_colors,test_descs=formatData(test_examples)
test_fourier_colors=applyFourierTransfrom(test_colors)

In [105]:
#get vocabulary
vocab=getVocab(train_descs)
vocab=["<PAD>"]+[START_SYMBOL, END_SYMBOL, UNK_SYMBOL]+vocab

#tok2id and id2tok implementation
tok2id={}
id2tok={}

for x,y in enumerate(vocab):
    tok2id[y]=x
    id2tok[x]=y

In [106]:
train_seqs=tokenizeSeqs(train_descs)
train_input_seqs,train_output_seqs=getInputAndTargetSeqs(train_seqs)

test_seqs=tokenizeSeqs(test_descs)
test_input_seqs,test_output_seqs=getInputAndTargetSeqs(test_seqs)

In [107]:
VOCAB_SIZE=len(vocab)
#model=createModel(HIDDEN_SIZE,EMBEDDING_SIZE,VOCAB_SIZE)
embedding_matrix=getEmbeddingMatrix(id2tok,EMBEDDING_SIZE)
model=createModel(HIDDEN_SIZE,EMBEDDING_SIZE,VOCAB_SIZE,COLOR_REP_SIZE,embedding_matrix)

In [108]:
def customLoss(yTrue,yPred):
    mask=tf.where(yTrue==0,0,1)
    mask=tf.cast(mask,dtype="float32")
    new_loss=tf.keras.losses.sparse_categorical_crossentropy(yTrue,yPred,from_logits=False)
    new_loss=new_loss*mask
    new_loss=tf.math.reduce_mean(new_loss,axis=1)
    final_loss=tf.math.reduce_mean(new_loss)
    return final_loss
    

# input_data=[train_fourier_colors[0:5].astype('float32'),train_input_seqs[0:5]]
# ref=model(input_data)
# k=train_output_seqs[0:5]
# customLoss(k,ref)

In [112]:
chkpnt=tf.keras.callbacks.ModelCheckpoint(
    filepath='/home/ravi/cs224u/models/checkpoint-{epoch:02d}-{val_loss:.4f}.hdf5', monitor='val_loss', 
    verbose=1, save_best_only=True,mode='auto', save_freq='epoch')

earlystopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto',restore_best_weights=True
)

lr_reducer=tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.8, patience=3, verbose=1, mode='auto',
    min_delta=0.0001, cooldown=3, min_lr=0)

model.compile(optimizer=tf.keras.optimizers.Nadam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [113]:
# model.load_weights("/home/ravi/cs224u/models/checkpoint-54-0.4332.hdf5")

In [114]:
model.fit([train_fourier_colors,train_input_seqs],train_output_seqs,
          validation_data=([test_fourier_colors,test_input_seqs],test_output_seqs),
          batch_size=2048,callbacks=[chkpnt,earlystopping,lr_reducer],max_queue_size=400,workers=8,
          use_multiprocessing=True,
          epochs=150)

Train on 42294 samples, validate on 4700 samples
Epoch 1/150
40960/42294 [============================>.] - ETA: 1s - loss: 3.6791 - accuracy: 0.6698
Epoch 00001: val_loss improved from inf to 1.46271, saving model to /home/ravi/cs224u/models/checkpoint-01-1.4627.hdf5
42294/42294 [==============================] - 35s 836us/sample - loss: 3.6102 - accuracy: 0.6719 - val_loss: 1.4627 - val_accuracy: 0.7329
Epoch 2/150
40960/42294 [============================>.] - ETA: 0s - loss: 1.2585 - accuracy: 0.7773
Epoch 00002: val_loss improved from 1.46271 to 1.12815, saving model to /home/ravi/cs224u/models/checkpoint-02-1.1282.hdf5
42294/42294 [==============================] - 28s 664us/sample - loss: 1.2543 - accuracy: 0.7781 - val_loss: 1.1282 - val_accuracy: 0.7994
Epoch 3/150
40960/42294 [============================>.] - ETA: 0s - loss: 1.0848 - accuracy: 0.8095
Epoch 00003: val_loss improved from 1.12815 to 1.04249, saving model to /home/ravi/cs224u/models/checkpoint-03-1.0425.hdf5
422

Epoch 24/150
40960/42294 [============================>.] - ETA: 0s - loss: 0.6994 - accuracy: 0.8577
Epoch 00024: val_loss improved from 0.72910 to 0.72484, saving model to /home/ravi/cs224u/models/checkpoint-24-0.7248.hdf5
42294/42294 [==============================] - 27s 639us/sample - loss: 0.6991 - accuracy: 0.8578 - val_loss: 0.7248 - val_accuracy: 0.8543
Epoch 25/150
40960/42294 [============================>.] - ETA: 0s - loss: 0.6946 - accuracy: 0.8581
Epoch 00025: val_loss improved from 0.72484 to 0.72242, saving model to /home/ravi/cs224u/models/checkpoint-25-0.7224.hdf5
42294/42294 [==============================] - 27s 650us/sample - loss: 0.6938 - accuracy: 0.8584 - val_loss: 0.7224 - val_accuracy: 0.8543
Epoch 26/150
40960/42294 [============================>.] - ETA: 0s - loss: 0.6890 - accuracy: 0.8588
Epoch 00026: val_loss improved from 0.72242 to 0.71946, saving model to /home/ravi/cs224u/models/checkpoint-26-0.7195.hdf5
42294/42294 [==============================] 

In [115]:
# model.load_weights("/home/ravi/cs224u/models/checkpoint-43-0.6984_baseline.hdf5")

In [116]:
PATH="/home/ravi/cs224u/models"

In [25]:
# with open("/home/ravi/codeBase/NLU/predictions.pickle","rb") as file:
#     id_predictions=pickle.load(file)
#     file.close()

In [26]:
# PATH="/home/ravi/codeBase/NLU/newimages"

In [117]:
from itertools import permutations 
perms=list(permutations([0,1,2]))
    
softmax=tf.keras.layers.Softmax()
def newgetProbabilityOfPredictions(color_rep,ids,model,limit):
    preds=model.predict([color_rep,ids])
    preds=softmax(preds)
    start_token_prob=np.zeros(VOCAB_SIZE)
    start_token_prob[tok2id['<s>']]=1
    probs = np.insert(preds, 0,start_token_prob,axis=1)
    new_probs=[i[:limit] for i in probs]
    return new_probs
   
def newlistenerAccuracy(colorrep,seq,model):
    seq=np.insert(seq,0,1)
    seq=seq[:-1]
    limit=len([x for x in seq if x>0])
    color_comb=[ np.array(colorrep[list(perm)]) for perm in perms]
    color_comb=np.array(color_comb)
    seqs= []+[seq]*len(perms)
    seqs=np.array(seqs)
    predictions=newgetProbabilityOfPredictions(color_comb,seqs,model,limit)
    final_probabilites=[]
    for comb in predictions:
        seq_probs=[x[i] for x,i in zip(comb,seq[:limit])]
        final_probabilites.append(seq_probs)
    perp = [np.prod(s) for s in final_probabilites]
    idx=np.argmin(perp)
    
    print(f"IDX : {perms[idx]}")
    if perms[idx][2]!=2:
        return 0
    else:
        return 1
    
newlistenerAccuracy(test_fourier_colors[4672],test_output_seqs[4672],model)

IDX : (1, 0, 2)


1

In [118]:
counter=[]
passed=[]
failed=[]
for i in range(test_fourier_colors.shape[0]):
    print("="*25)
    print(f"ID: {i}")
    val=listenerAccuracy(test_fourier_colors[i],test_output_seqs[i],model)
    counter.append(val)
    if val==0:
        print("Failed")
        failed.append(i)
    else:
        print("Passed")
        passed.append(i)

ID: 0


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in power


Perm: (1, 2, 0)
Failed
ID: 1
Perm: (0, 2, 1)
Failed
ID: 2
Perm: (0, 2, 1)
Failed
ID: 3
Perm: (2, 1, 0)
Failed
ID: 4
Perm: (0, 1, 2)
Passed
ID: 5
Perm: (0, 1, 2)
Passed
ID: 6
Perm: (1, 2, 0)
Failed
ID: 7
Perm: (0, 2, 1)
Failed
ID: 8
Perm: (0, 2, 1)
Failed
ID: 9
Perm: (1, 2, 0)
Failed
ID: 10
Perm: (1, 0, 2)
Passed
ID: 11
Perm: (0, 1, 2)
Passed
ID: 12
Perm: (1, 2, 0)
Failed
ID: 13
Perm: (1, 0, 2)
Passed
ID: 14
Perm: (0, 2, 1)
Failed
ID: 15
Perm: (0, 2, 1)
Failed
ID: 16
Perm: (0, 2, 1)
Failed
ID: 17
Perm: (0, 1, 2)
Passed
ID: 18
Perm: (2, 0, 1)
Failed
ID: 19
Perm: (2, 0, 1)
Failed
ID: 20
Perm: (0, 2, 1)
Failed
ID: 21
Perm: (2, 1, 0)
Failed
ID: 22
Perm: (0, 2, 1)
Failed
ID: 23
Perm: (0, 2, 1)
Failed
ID: 24
Perm: (0, 2, 1)
Failed
ID: 25
Perm: (0, 1, 2)
Passed
ID: 26
Perm: (0, 2, 1)
Failed
ID: 27
Perm: (1, 0, 2)
Passed
ID: 28
Perm: (0, 2, 1)
Failed
ID: 29
Perm: (0, 2, 1)
Failed
ID: 30
Perm: (1, 0, 2)
Passed
ID: 31
Perm: (0, 2, 1)
Failed
ID: 32
Perm: (0, 1, 2)
Passed
ID: 33
Perm: (1, 0, 2)
Pas

Perm: (0, 1, 2)
Passed
ID: 148
Perm: (0, 2, 1)
Failed
ID: 149
Perm: (0, 1, 2)
Passed
ID: 150
Perm: (1, 0, 2)
Passed
ID: 151
Perm: (1, 0, 2)
Passed
ID: 152
Perm: (2, 0, 1)
Failed
ID: 153
Perm: (0, 2, 1)
Failed
ID: 154
Perm: (0, 2, 1)
Failed
ID: 155
Perm: (1, 2, 0)
Failed
ID: 156
Perm: (0, 1, 2)
Passed
ID: 157
Perm: (0, 2, 1)
Failed
ID: 158
Perm: (0, 1, 2)
Passed
ID: 159
Perm: (2, 0, 1)
Failed
ID: 160
Perm: (1, 2, 0)
Failed
ID: 161
Perm: (0, 1, 2)
Passed
ID: 162
Perm: (0, 2, 1)
Failed
ID: 163
Perm: (0, 2, 1)
Failed
ID: 164
Perm: (2, 0, 1)
Failed
ID: 165
Perm: (0, 2, 1)
Failed
ID: 166
Perm: (2, 0, 1)
Failed
ID: 167
Perm: (1, 2, 0)
Failed
ID: 168
Perm: (1, 0, 2)
Passed
ID: 169
Perm: (1, 2, 0)
Failed
ID: 170
Perm: (0, 1, 2)
Passed
ID: 171
Perm: (0, 1, 2)
Passed
ID: 172
Perm: (2, 0, 1)
Failed
ID: 173
Perm: (1, 0, 2)
Passed
ID: 174
Perm: (1, 2, 0)
Failed
ID: 175
Perm: (0, 1, 2)
Passed
ID: 176
Perm: (1, 0, 2)
Passed
ID: 177
Perm: (0, 1, 2)
Passed
ID: 178
Perm: (0, 2, 1)
Failed
ID: 179
Perm: (0

Perm: (0, 2, 1)
Failed
ID: 292
Perm: (0, 1, 2)
Passed
ID: 293
Perm: (0, 2, 1)
Failed
ID: 294
Perm: (0, 1, 2)
Passed
ID: 295
Perm: (0, 1, 2)
Passed
ID: 296
Perm: (1, 0, 2)
Passed
ID: 297
Perm: (2, 0, 1)
Failed
ID: 298
Perm: (2, 1, 0)
Failed
ID: 299
Perm: (1, 0, 2)
Passed
ID: 300
Perm: (0, 2, 1)
Failed
ID: 301
Perm: (0, 2, 1)
Failed
ID: 302
Perm: (0, 1, 2)
Passed
ID: 303
Perm: (2, 1, 0)
Failed
ID: 304
Perm: (0, 2, 1)
Failed
ID: 305
Perm: (0, 1, 2)
Passed
ID: 306
Perm: (0, 1, 2)
Passed
ID: 307
Perm: (0, 1, 2)
Passed
ID: 308
Perm: (1, 0, 2)
Passed
ID: 309
Perm: (0, 1, 2)
Passed
ID: 310
Perm: (0, 2, 1)
Failed
ID: 311
Perm: (0, 2, 1)
Failed
ID: 312
Perm: (1, 2, 0)
Failed
ID: 313
Perm: (0, 1, 2)
Passed
ID: 314
Perm: (0, 1, 2)
Passed
ID: 315
Perm: (1, 0, 2)
Passed
ID: 316
Perm: (1, 0, 2)
Passed
ID: 317
Perm: (1, 0, 2)
Passed
ID: 318
Perm: (0, 1, 2)
Passed
ID: 319
Perm: (1, 2, 0)
Failed
ID: 320
Perm: (0, 2, 1)
Failed
ID: 321
Perm: (1, 0, 2)
Passed
ID: 322
Perm: (0, 2, 1)
Failed
ID: 323
Perm: (1

...

Perm: (1, 0, 2)
Passed
ID: 3995
Perm: (1, 0, 2)
Passed
ID: 3996
Perm: (0, 2, 1)
Failed
ID: 3997
Perm: (0, 1, 2)
Passed
ID: 3998
Perm: (1, 0, 2)
Passed
ID: 3999
Perm: (1, 0, 2)
Passed
ID: 4000
Perm: (2, 0, 1)
Failed
ID: 4001
Perm: (0, 2, 1)
Failed
ID: 4002
Perm: (2, 1, 0)
Failed
ID: 4003
Perm: (0, 2, 1)
Failed
ID: 4004
Perm: (2, 0, 1)
Failed
ID: 4005
Perm: (1, 2, 0)
Failed
ID: 4006
Perm: (0, 2, 1)
Failed
ID: 4007
Perm: (0, 1, 2)
Passed
ID: 4008
Perm: (1, 0, 2)
Passed
ID: 4009
Perm: (1, 0, 2)
Passed
ID: 4010
Perm: (1, 2, 0)
Failed
ID: 4011
Perm: (0, 1, 2)
Passed
ID: 4012
Perm: (1, 2, 0)
Failed
ID: 4013
Perm: (2, 1, 0)
Failed
ID: 4014
Perm: (0, 2, 1)
Failed
ID: 4015
Perm: (0, 1, 2)
Passed
ID: 4016
Perm: (0, 1, 2)
Passed
ID: 4017
Perm: (0, 1, 2)
Passed
ID: 4018
Perm: (0, 1, 2)
Passed
ID: 4019
Perm: (0, 1, 2)
Passed
ID: 4020
Perm: (1, 2, 0)
Failed
ID: 4021
Perm: (0, 1, 2)
Passed
ID: 4022
Perm: (0, 2, 1)
Failed
ID: 4023
Perm: (0, 1, 2)
Passed
ID: 4024
Perm: (0, 2, 1)
Failed
ID: 4025
Perm: (0

Perm: (0, 2, 1)
Failed
ID: 4137
Perm: (0, 1, 2)
Passed
ID: 4138
Perm: (0, 1, 2)
Passed
ID: 4139
Perm: (1, 0, 2)
Passed
ID: 4140
Perm: (0, 1, 2)
Passed
ID: 4141
Perm: (0, 1, 2)
Passed
ID: 4142
Perm: (1, 0, 2)
Passed
ID: 4143
Perm: (1, 2, 0)
Failed
ID: 4144
Perm: (0, 2, 1)
Failed
ID: 4145
Perm: (0, 1, 2)
Passed
ID: 4146
Perm: (0, 1, 2)
Passed
ID: 4147
Perm: (0, 2, 1)
Failed
ID: 4148
Perm: (0, 2, 1)
Failed
ID: 4149
Perm: (0, 1, 2)
Passed
ID: 4150
Perm: (1, 0, 2)
Passed
ID: 4151
Perm: (0, 1, 2)
Passed
ID: 4152
Perm: (0, 1, 2)
Passed
ID: 4153
Perm: (1, 2, 0)
Failed
ID: 4154
Perm: (0, 2, 1)
Failed
ID: 4155
Perm: (1, 0, 2)
Passed
ID: 4156
Perm: (0, 2, 1)
Failed
ID: 4157
Perm: (0, 1, 2)
Passed
ID: 4158
Perm: (1, 0, 2)
Passed
ID: 4159
Perm: (0, 2, 1)
Failed
ID: 4160
Perm: (1, 2, 0)
Failed
ID: 4161
Perm: (0, 2, 1)
Failed
ID: 4162
Perm: (1, 2, 0)
Failed
ID: 4163
Perm: (2, 1, 0)
Failed
ID: 4164
Perm: (2, 0, 1)
Failed
ID: 4165
Perm: (1, 0, 2)
Passed
ID: 4166
Perm: (1, 0, 2)
Passed
ID: 4167
Perm: (1

Perm: (0, 1, 2)
Passed
ID: 4279
Perm: (1, 2, 0)
Failed
ID: 4280
Perm: (0, 1, 2)
Passed
ID: 4281
Perm: (1, 0, 2)
Passed
ID: 4282
Perm: (0, 1, 2)
Passed
ID: 4283
Perm: (0, 1, 2)
Passed
ID: 4284
Perm: (0, 2, 1)
Failed
ID: 4285
Perm: (0, 1, 2)
Passed
ID: 4286
Perm: (0, 1, 2)
Passed
ID: 4287
Perm: (0, 1, 2)
Passed
ID: 4288
Perm: (2, 1, 0)
Failed
ID: 4289
Perm: (1, 2, 0)
Failed
ID: 4290
Perm: (1, 0, 2)
Passed
ID: 4291
Perm: (0, 2, 1)
Failed
ID: 4292
Perm: (0, 2, 1)
Failed
ID: 4293
Perm: (0, 1, 2)
Passed
ID: 4294
Perm: (0, 2, 1)
Failed
ID: 4295
Perm: (2, 0, 1)
Failed
ID: 4296
Perm: (0, 1, 2)
Passed
ID: 4297
Perm: (0, 1, 2)
Passed
ID: 4298
Perm: (1, 0, 2)
Passed
ID: 4299
Perm: (0, 2, 1)
Failed
ID: 4300
Perm: (0, 2, 1)
Failed
ID: 4301
Perm: (1, 0, 2)
Passed
ID: 4302
Perm: (0, 1, 2)
Passed
ID: 4303
Perm: (1, 0, 2)
Passed
ID: 4304
Perm: (1, 0, 2)
Passed
ID: 4305
Perm: (1, 2, 0)
Failed
ID: 4306
Perm: (1, 2, 0)
Failed
ID: 4307
Perm: (2, 1, 0)
Failed
ID: 4308
Perm: (0, 1, 2)
Passed
ID: 4309
Perm: (0

Perm: (0, 1, 2)
Passed
ID: 4421
Perm: (0, 1, 2)
Passed
ID: 4422
Perm: (0, 1, 2)
Passed
ID: 4423
Perm: (0, 1, 2)
Passed
ID: 4424
Perm: (2, 0, 1)
Failed
ID: 4425
Perm: (0, 1, 2)
Passed
ID: 4426
Perm: (0, 1, 2)
Passed
ID: 4427
Perm: (0, 2, 1)
Failed
ID: 4428
Perm: (0, 1, 2)
Passed
ID: 4429
Perm: (0, 2, 1)
Failed
ID: 4430
Perm: (0, 1, 2)
Passed
ID: 4431
Perm: (0, 2, 1)
Failed
ID: 4432
Perm: (1, 2, 0)
Failed
ID: 4433
Perm: (0, 1, 2)
Passed
ID: 4434
Perm: (0, 1, 2)
Passed
ID: 4435
Perm: (1, 0, 2)
Passed
ID: 4436
Perm: (1, 0, 2)
Passed
ID: 4437
Perm: (0, 2, 1)
Failed
ID: 4438
Perm: (2, 0, 1)
Failed
ID: 4439
Perm: (0, 1, 2)
Passed
ID: 4440
Perm: (1, 0, 2)
Passed
ID: 4441
Perm: (0, 2, 1)
Failed
ID: 4442
Perm: (0, 1, 2)
Passed
ID: 4443
Perm: (1, 2, 0)
Failed
ID: 4444
Perm: (0, 1, 2)
Passed
ID: 4445
Perm: (0, 1, 2)
Passed
ID: 4446
Perm: (1, 0, 2)
Passed
ID: 4447
Perm: (0, 2, 1)
Failed
ID: 4448
Perm: (1, 0, 2)
Passed
ID: 4449
Perm: (1, 0, 2)
Passed
ID: 4450
Perm: (0, 2, 1)
Failed
ID: 4451
Perm: (0

Perm: (0, 1, 2)
Passed
ID: 4563
Perm: (1, 0, 2)
Passed
ID: 4564
Perm: (0, 2, 1)
Failed
ID: 4565
Perm: (1, 0, 2)
Passed
ID: 4566
Perm: (1, 0, 2)
Passed
ID: 4567
Perm: (1, 0, 2)
Passed
ID: 4568
Perm: (1, 0, 2)
Passed
ID: 4569
Perm: (0, 1, 2)
Passed
ID: 4570
Perm: (0, 1, 2)
Passed
ID: 4571
Perm: (2, 0, 1)
Failed
ID: 4572
Perm: (0, 1, 2)
Passed
ID: 4573
Perm: (0, 1, 2)
Passed
ID: 4574
Perm: (0, 1, 2)
Passed
ID: 4575
Perm: (1, 2, 0)
Failed
ID: 4576
Perm: (1, 0, 2)
Passed
ID: 4577
Perm: (0, 1, 2)
Passed
ID: 4578
Perm: (0, 2, 1)
Failed
ID: 4579
Perm: (2, 1, 0)
Failed
ID: 4580
Perm: (1, 0, 2)
Passed
ID: 4581
Perm: (2, 1, 0)
Failed
ID: 4582
Perm: (1, 0, 2)
Passed
ID: 4583
Perm: (0, 1, 2)
Passed
ID: 4584
Perm: (0, 1, 2)
Passed
ID: 4585
Perm: (1, 0, 2)
Passed
ID: 4586
Perm: (0, 2, 1)
Failed
ID: 4587
Perm: (0, 1, 2)
Passed
ID: 4588
Perm: (0, 2, 1)
Failed
ID: 4589
Perm: (0, 1, 2)
Passed
ID: 4590
Perm: (1, 0, 2)
Passed
ID: 4591
Perm: (1, 0, 2)
Passed
ID: 4592
Perm: (0, 2, 1)
Failed
ID: 4593
Perm: (1

In [37]:
print(f"Listener Accuracy : {sum(counter)/len(counter)}")

Listener Accuracy : 0.5270212765957447


In [38]:
len(counter)-sum(counter)

2223

In [ ]:
def inference(colors,model,max_steps):
    colors=colors.astype('float32')
    batch_size=colors.shape[0]
    start_token_id=tok2id['<s>']
    predictions=np.full((1,MAX_LEN),start_token_id)
    for i in range(0,max_steps):
        preds=model([colors,predictions])
        pred_toks=tf.argmax(preds[0][i])
        predictions[0][i+1]=pred_toks
        if pred_toks==2:
            return predictions[0][:i+2]
    return predictions[0]

# color_rep=np.expand_dims(test_fourier_colors[48],axis=0)
# pred=inference(color_rep,model,MAX_STEPS)
# print(pred)

In [ ]:
MAX_STEPS=10
id_predictions=[]
for i in tqdm(range(test_fourier_colors.shape[0])):
    color_rep=np.expand_dims(test_fourier_colors[i],axis=0)
    pred=inference(color_rep,model,MAX_STEPS)
    id_predictions.append(pred)

In [ ]:
with open(PATH+"/predictions_better.pickle","wb") as file:
    pickle.dump(id_predictions,file)
    file.close()

In [ ]:
for i in zip(test_input_seqs,id_predictions,range(len(test_examples)),counter):
    orig_seq=[id2tok[x] for x in i[0] if x>2]
    pred_seq=[id2tok[x] for x in i[1] if x>2 ]
#     test_examples[i[2]].savePicture(predicted_text,i[2],PATH)
    if i[3]!=1:
        print("="*50)
        print(i[1])
        print(f"ID:{i[2]}")
        print(f"Result:{'Passed' if i[3]==1 else 'Failed'}")
        print(f"Type: {test_examples[i[2]].condition}")
        print(f"Original sequence: {' '.join(orig_seq)}")
        print(f"Predicted sequence: {' '.join(pred_seq)}")

In [ ]:
test_examples[732].display()

In [ ]:
X=[]
for i in test_input_seqs:
    X.append([id2tok[int(x)] for x in i if x>2])

Y=[]
for i in id_predictions:
    Y.append([ id2tok[int(x)] for x in i if x>2])


In [ ]:
score=corpus_bleu(X,Y, weights=(1,))
print(f"The corpus BLEU Score is : {score}")